In [55]:
# Libraries
import pandas as pd
import numpy as np
import time
import datetime as dt
from datetime import timedelta
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn import linear_model
import sys
from keras.layers import Bidirectional
import io
import cv2
from ipywidgets import FileUpload, Label, Image, Button, Output

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import load_model

In [56]:
!git clone https://github.com/msabelhaus/load-forecasting.git

fatal: destination path 'load-forecasting' already exists and is not an empty directory.


In [57]:
lstm_model = load_model('load-forecasting/loadFcastLSTM.h5')

In [58]:
# Upload CSV
csv_upload = FileUpload(accept='*.csv', multiple=False)
csv_label = Label(value = "Please upload a single csv below.")

display(csv_label, csv_upload)

Label(value='Please upload a single csv below.')

FileUpload(value={}, accept='*.csv', description='Upload')

In [61]:
input_file = list(csv_upload.value.values())[0]
content = input_file['content']
content = io.StringIO(content.decode('utf-8'))
df = pd.read_csv(content)

predict_button = Button(description="Predict!")
result = Output()

In [62]:
LoadMW_Plus1 = df["LoadMW"].shift(periods=-1)
LoadMW_Plus1.name = 'LoadMW_Plus1'
df = pd.concat([df, LoadMW_Plus1], axis=1)

In [63]:
null_index = df['LoadMW_Plus1'].isnull()
null_index

0    False
1    False
2    False
3    False
4    False
5    False
6    False
7    False
8    False
9     True
Name: LoadMW_Plus1, dtype: bool

In [64]:
def predict(button_info):
    try:
        df_scaled = df.copy()

        # Initialize scaler (here we use Min/Max)
        sc = MinMaxScaler()

        # Fit
        sc_x = StandardScaler().fit(df_scaled[["TempF","LoadMW"]])
        sc_y = StandardScaler().fit(df_scaled.loc[~null_index, ['LoadMW_Plus1']].values.reshape(-1,1))

        # Scale numerical variables
        df_scaled[["TempF","LoadMW"]] = sc_x.transform(df_scaled[["TempF","LoadMW"]])
        df_scaled.loc[~null_index, ["LoadMW_Plus1"]] = sc_y.transform(df_scaled.loc[~null_index, ['LoadMW_Plus1']].values.reshape(-1,1))

        # Remove time objects from scaled dataframe
        df_scaled = df_scaled[["Peak_Plus1","TempF","LoadMW","LoadMW_Plus1"]]

        # Reshape dataframe 
        # Put our timeseries variables into arrays and horizontally stack them
        # This input format is necessary to be able to run through the split_sequences() function
        in_seq1 = np.array(df_scaled["LoadMW"])
        in_seq2 = np.array(df_scaled["TempF"])
        in_seq3 = np.array(df_scaled["Peak_Plus1"])
        out_seq = np.array(df_scaled["LoadMW_Plus1"])

        # convert to [rows, columns] structure
        in_seq1 = in_seq1.reshape((len(in_seq1), 1))
        in_seq2 = in_seq2.reshape((len(in_seq2), 1))
        in_seq3 = in_seq3.reshape((len(in_seq3), 1)) 
        out_seq = out_seq.reshape((len(out_seq), 1))

        # horizontally stack columns
        dataset = np.hstack((in_seq1, in_seq2, in_seq3, out_seq))

        # The function below splits our dataset into samples of sequences of time stpes
        # It is taken from machinelearningmastery
        def split_sequences(sequences, n_steps):
          X, y = list(), list()
          for i in range(len(sequences)):
            # find the end of this pattern
            end_ix = i + n_steps
            # check if we are beyond the dataset
            if end_ix > len(sequences):
              break
            # gather input and output parts of the pattern
            seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
            X.append(seq_x)
            y.append(seq_y)
          return np.array(X), np.array(y)


        X, y = split_sequences(dataset, 9)

        prediction = sc_y.inverse_transform(lstm_model.predict(X[1].reshape(1,9,3))).item()

        with result:
            print(f"The load forecast for the next hour is {prediction}.")
    except Exception as e:
        with result:
            print(e)

In [65]:
predict_button.on_click(predict)
display(predict_button, result)

Button(description='Predict!', style=ButtonStyle())

Output()